# Estudio de los datos en el curated stage
### Ayuda al desarrollo de la etl


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import *
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::114652167878:role/AWSGlueAndS3RoleGrupo2
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 29303e97-db50-4e33-aad1-030c368b864c
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 29303e97

### Extract data

In [2]:
df_dim_users = (
    spark.read
    .format("parquet")
    .load("s3://pr2-grupo3-rodaan-curated-layer/data/dim_users/")
)
df_dim_users.show()

+-----------+------+----+------------------+------------+--------------------+-------------+
|        #id|gender| age|           country|  registered|        country_name|register_date|
+-----------+------+----+------------------+------------+--------------------+-------------+
|user_000001|     m|null|             Japan|Aug 13, 2006|               Japan|   2006-08-13|
|user_000002|     f|null|              Peru|Feb 24, 2006|                Peru|   2006-02-24|
|user_000003|     m|  22|     United States|Oct 30, 2005|United States of ...|   2005-10-30|
|user_000004|     f|null|              null|Apr 26, 2006|                null|   2006-04-26|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|            Bulgaria|   2006-06-29|
|user_000006|  null|  24|Russian Federation|May 18, 2006|              Russia|   2006-05-18|
|user_000007|     f|null|     United States|Jan 22, 2006|United States of ...|   2006-01-22|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|            S

In [13]:
df_dim_countries = (
    spark.read
    .format("parquet")
    .load("s3://pr2-grupo3-rodaan-curated-layer/data/dim_countries/")
)
df_dim_countries.show()

+--------------------+--------------------+-------------------+--------------------+
|                pais|             country|         continente|           continent|
+--------------------+--------------------+-------------------+--------------------+
|             Albania|             Albania|             Europa|              Europe|
|            Alemania|             Germany|             Europa|              Europe|
|            Barbados|            Barbados|            América|             America|
|            Bulgaria|            Bulgaria|             Europa|              Europe|
|             Burundi|             Burundi|             África|              Africa|
|             Camerún|            Cameroon|             África|              Africa|
|Emiratos Árabes U...|United Arab Emirates|               Asia|                Asia|
|    Guayana Francesa|       French Guiana|            América|             America|
|       Guinea-Bissau|       Guinea-Bissau|             África|  

In [4]:
df_dim_horoscopes = (
    spark.read
    .format("parquet")
    .load("s3://pr2-grupo3-rodaan-curated-layer/data/dim_horoscopes/")
)
df_dim_horoscopes.show()

+---------+------+---+-----+
|Horoscope|  Date|Day|Month|
+---------+------+---+-----+
|Capricorn| 1-Jan|  1|    1|
|Capricorn| 2-Jan|  2|    1|
|Capricorn| 3-Jan|  3|    1|
|Capricorn| 4-Jan|  4|    1|
|Capricorn| 5-Jan|  5|    1|
|Capricorn| 6-Jan|  6|    1|
|Capricorn| 7-Jan|  7|    1|
|Capricorn| 8-Jan|  8|    1|
|Capricorn| 9-Jan|  9|    1|
|Capricorn|10-Jan| 10|    1|
|Capricorn|11-Jan| 11|    1|
|Capricorn|12-Jan| 12|    1|
|Capricorn|13-Jan| 13|    1|
|Capricorn|14-Jan| 14|    1|
|Capricorn|15-Jan| 15|    1|
|Capricorn|16-Jan| 16|    1|
|Capricorn|17-Jan| 17|    1|
|Capricorn|18-Jan| 18|    1|
|Capricorn|19-Jan| 19|    1|
| Aquarius|20-Jan| 20|    1|
+---------+------+---+-----+
only showing top 20 rows


In [6]:
df_dim_facts = (
    spark.read
    .format("parquet")
    .load("s3://pr2-grupo3-rodaan-curated-layer/data/facts_user/")
)
df_dim_facts.show(truncate=False)

+--------------------+------------------------------------+------------------------+------------------------------------+-------------------------------+-------------------+-----------+
|fact_date           |group_id                            |group_name              |song_id                             |song_name                      |fact_timestamp     |user_id    |
+--------------------+------------------------------------+------------------------+------------------------------------+-------------------------------+-------------------+-----------+
|2007-09-29T21:23:06Z|277e21a9-2d64-452d-96c4-2d23a7af5891|Mr. Bungle              |2fcb70a7-676b-405f-93d1-ed22b67e13af|The Girls Of Porn              |2007-09-29 21:23:06|user_000332|
|2007-01-23T23:42:33Z|84dc4f23-c0b8-4fe1-bbca-a3993ddc8fc2|Primus                  |921832e4-f836-4974-9aec-bcdb05fafe97|Los Bastardos                  |2007-01-23 23:42:33|user_000332|
|2009-06-16T04:12:24Z|d43d12a1-2dc9-4257-a2fd-0a3bb1081b86|Johnny Cash

#### Cambio de nombre a la columna id de usuario

In [8]:
df_dim_users_rename_id = (
    df_dim_users
    .withColumnRenamed("#id","user_id")
)

#### Añadir columnas de mes y dia de registro
#### El cliente necesita saber a que horocopo pertenece cada usuario en base a su fecha de registro
#### Estas columnas serán las encargadas de hacer el cruce

In [9]:
def day(date):
    date_str = str(date)
    date_split = date_str.split("-")
    day = int(date_split[2])
    return day

dayUDF = udf(lambda date: day(date))

df_dim_users_month_day_register = (
    df_dim_users_rename_id
    .withColumn("register_month", month("register_date"))
    .withColumn("register_day", dayUDF("register_date"))    
)
df_dim_users_month_day_register.show()

+-----------+------+----+------------------+------------+--------------------+-------------+--------------+------------+
|    user_id|gender| age|           country|  registered|        country_name|register_date|register_month|register_day|
+-----------+------+----+------------------+------------+--------------------+-------------+--------------+------------+
|user_000001|     m|null|             Japan|Aug 13, 2006|               Japan|   2006-08-13|             8|          13|
|user_000002|     f|null|              Peru|Feb 24, 2006|                Peru|   2006-02-24|             2|          24|
|user_000003|     m|  22|     United States|Oct 30, 2005|United States of ...|   2005-10-30|            10|          30|
|user_000004|     f|null|              null|Apr 26, 2006|                null|   2006-04-26|             4|          26|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|            Bulgaria|   2006-06-29|             6|          29|
|user_000006|  null|  24|Russian

#### Cruce para conocer el horoscopo de cada usuario
#### El cliente quiere conocerlo por lo que es buena idea agregarlo a la dimension de usuarios directamente para ahorrar cruces a la hora de hacer analitica

In [10]:
df_users_join_horoscopes = (
    df_dim_users_month_day_register
    .join(df_dim_horoscopes, (df_dim_users_month_day_register.register_day == df_dim_horoscopes.Day) & (df_dim_users_month_day_register.register_month == df_dim_horoscopes.Month),"left")
    .drop("Date","Day","Month")
    .withColumnRenamed("Horocope","horoscope")  
)

#### Comprobar que todos los usuarios quedan enmarcados en un horoscopo

In [11]:
(
    df_dim_users_month_day_register
    .join(df_dim_horoscopes, (df_dim_users_month_day_register.register_day == df_dim_horoscopes.Day) & (df_dim_users_month_day_register.register_month == df_dim_horoscopes.Month),"left")
    .drop("Date","Day","Month")
    .withColumnRenamed("Horocope","horoscope")
    .where("horoscope is null")
    .show()  
)

+-------+------+---+-------+----------+------------+-------------+--------------+------------+---------+
|user_id|gender|age|country|registered|country_name|register_date|register_month|register_day|Horoscope|
+-------+------+---+-------+----------+------------+-------------+--------------+------------+---------+
+-------+------+---+-------+----------+------------+-------------+--------------+------------+---------+


#### Cruce para añadir el continente del ususario
#### El cliente lo requiere, por lo que seria util agregarlo a la dimension de usuarios en lugar de cruzar constantemente

In [14]:
df_dim_users_with_continent = (
    df_dim_users_month_day_register
    .join(df_dim_countries, df_dim_users_month_day_register.country_name == df_dim_countries.country, "left")
    .drop(df_dim_countries.country)
    
)
df_dim_users_with_continent.show()

+-----------+------+----+------------------+------------+--------------------+-------------+--------------+------------+--------------------+----------+---------+
|    user_id|gender| age|           country|  registered|        country_name|register_date|register_month|register_day|                pais|continente|continent|
+-----------+------+----+------------------+------------+--------------------+-------------+--------------+------------+--------------------+----------+---------+
|user_000001|     m|null|             Japan|Aug 13, 2006|               Japan|   2006-08-13|             8|          13|               Japón|      Asia|     Asia|
|user_000002|     f|null|              Peru|Feb 24, 2006|                Peru|   2006-02-24|             2|          24|                Perú|   América|  America|
|user_000003|     m|  22|     United States|Oct 30, 2005|United States of ...|   2005-10-30|            10|          30|Estados Unidos de...|   América|  America|
|user_000004|     f|nu

In [15]:
df_dim_user_refined = (
    df_dim_users_with_continent
    .drop("nombre")
)

### Hechos de usuarios

### La asignacion del id del grupo y el nombre del grupo no es 1 a 1, por lo que el campo no es fiable para usarlo de identificador unico

In [16]:
df_dim_groups = (
    df_dim_facts
    .select("group_name","group_id")
    .distinct()
    
    
)
count = (
    df_dim_facts
    .select("group_name")
    
    .distinct()
    .count()
    
)
print(f"Cuenta por group name {count} y cuante por los dos campos {df_dim_groups.count()}")
df_dim_facts.where( col("group_id").isNull()).show(truncate=False)


Cuenta por group name 174090 y cuante por los dos campos 177023
+--------------------+--------+----------------------------+-------+------------------------------------------+-------------------+-----------+
|fact_date           |group_id|group_name                  |song_id|song_name                                 |fact_timestamp     |user_id    |
+--------------------+--------+----------------------------+-------+------------------------------------------+-------------------+-----------+
|2009-06-13T21:11:50Z|null    |Os_Nerds                    |null   |Me Dê @Op No #Seu_Coracao                 |2009-06-13 21:11:50|user_000349|
|2009-06-13T13:38:34Z|null    |The Blue Velvets - Golliwogs|null   |Come On Baby                              |2009-06-13 13:38:34|user_000349|
|2009-06-13T02:40:37Z|null    |Nikolo Ktozev, Nostradamus  |null   |Chosen Man                                |2009-06-13 02:40:37|user_000349|
|2009-06-13T00:05:31Z|null    |Banda Clandestino           |null   |Ñemi

In [9]:
(
    df_dim_facts
    .groupBy("group_name","song_name")
    .agg(countDistinct("group_name","song_name").alias("cuenta"))
    .where(col("cuenta")>1)

    .show(truncate=False)
)

+----------+---------+------+
|group_name|song_name|cuenta|
+----------+---------+------+
+----------+---------+------+


In [7]:
(
    df_dim_facts
    .select("group_name","song_name","group_id","song_id")
    .distinct()
    .count()
)

1493185


#### Todos los hechos vienen bien identificados por el user id

In [13]:
(
    df_dim_facts
    .where(col("user_id").isNull())
    .show(truncate=False)
)

+---------+--------+----------+-------+---------+--------------+----+-----+-------+
|fact_date|group_id|group_name|song_id|song_name|fact_timestamp|year|month|user_id|
+---------+--------+----------+-------+---------+--------------+----+-----+-------+
+---------+--------+----------+-------+---------+--------------+----+-----+-------+
